<a href="https://colab.research.google.com/github/AtienBen/Mask_map/blob/main/Mask_map_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 25 23:13:40 2023

@author: Atien
"""

import requests
import json
import sqlite3
import datetime



# 利用 requests 對 API 來源發送一個請求
url = ("https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json")
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
str_a = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(str_a)

med_count = {}
# 填入欄位名稱
for d in data['features']:
    #conunty = d['properties']['county'] 部分資料此欄位空缺
    conunty = d['properties']['address'][:3]
    if conunty not in med_count:
         med_count[conunty] = 0
    else:
         med_count[conunty] +=1

print(med_count)

mask_count = {}

# 填入欄位名稱
for d in data['features']:
    conunty = d['properties']['address'][:3]
    mask_adult_count = d['properties']['mask_adult']
    if conunty not in mask_count:
         mask_count[conunty] = 0
    else:
         mask_count[conunty] += mask_adult_count

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1]))

print(mask_count)

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增pharmacies且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

#新增資料med_count至資料庫
for city,counts in med_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}' , '{counts}' , '{t}')")
  conn.commit()

# 新增masks且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks
             (city text, mask_adult_count text, createdAt datetime)''')
c.execute('''DELETE FROM masks''')
conn.commit()

#新增資料mask_count至資料庫
for city,mask_adult_count in mask_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO masks VALUES ('{city}' , '{mask_adult_count}' , '{t}')")
  conn.commit()

# 查詢資料FROM pharmacies
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())
# 查詢資料FROM masks
c.execute("SELECT * FROM masks")
print(c.fetchall())


conn.commit()
conn.close()

{'臺北市': 339, '高雄市': 423, '臺中市': 435, '臺南市': 279, '基隆市': 56, '新竹市': 38, '嘉義市': 66, '新北市': 506, '桃園市': 270, '新竹縣': 47, '宜蘭縣': 75, '苗栗縣': 56, '彰化縣': 180, '南投縣': 66, '雲林縣': 130, '嘉義縣': 83, '屏東縣': 139, '澎湖縣': 10, '花蓮縣': 46, '臺東縣': 22, '金門縣': 5, '連江縣': 0}
{'連江縣': 0, '澎湖縣': 11350, '金門縣': 15510, '臺東縣': 27720, '新竹市': 45630, '新竹縣': 53690, '花蓮縣': 56570, '基隆市': 79840, '南投縣': 81680, '宜蘭縣': 105830, '嘉義市': 105870, '嘉義縣': 112190, '苗栗縣': 122600, '屏東縣': 181480, '雲林縣': 198020, '彰化縣': 218090, '臺北市': 323660, '桃園市': 343670, '臺南市': 406930, '高雄市': 428940, '臺中市': 555160, '新北市': 677540}
[('臺北市', '339', '2023-05-25 15:24:40.570951'), ('高雄市', '423', '2023-05-25 15:24:40.578807'), ('臺中市', '435', '2023-05-25 15:24:40.586931'), ('臺南市', '279', '2023-05-25 15:24:40.596302'), ('基隆市', '56', '2023-05-25 15:24:40.604789'), ('新竹市', '38', '2023-05-25 15:24:40.615144'), ('嘉義市', '66', '2023-05-25 15:24:40.623430'), ('新北市', '506', '2023-05-25 15:24:40.631748'), ('桃園市', '270', '2023-05-25 15:24:40.640924'), ('新竹縣', '47', '2023-